## Constrastive Loss
主要用于拉近相似样本的embedding距离，推远不相似样本距离。

#### 对比损失定义
$$ L(x_i, x_j, y_{ij}) = y_{ij} \cdot D^2 + (1-y_{ij}) \cdot \max(0, m-D)^2
 $$

In [2]:
import torch
import torch.nn as nn

/Users/xhades/Documents/github/LLM101/.venv/lib/python3.9/site-packages/torch/_subclasses/functional_tensor.py:275: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [3]:

class ContrastiveLoss(nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        """
        参数:
        output1: 第一个样本的特征向量 shape=(batch_size, feature_dim)
        output2: 第二个样本的特征向量 shape=(batch_size, feature_dim)
        label: 相似性标签 (1=相似, 0=不相似) shape=(batch_size,)
        """
        # 计算欧氏距离
        euclidean_distance = torch.nn.functional.pairwise_distance(output1, output2)
        
        # 对于相似样本(y=1): loss = distance^2
        # 对于不相似样本(y=0): loss = max(0, margin - distance)^2
        loss_contrastive = torch.mean(
            label * torch.pow(euclidean_distance, 2) +  # 相似样本的损失
            (1 - label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2)  # 不相似样本的损失
        )
        
        return loss_contrastive


In [8]:
### inference
# 使用示例
batch_size = 2
feature_dim = 128

# 随机生成样本数据
output1 = torch.randn(batch_size, feature_dim)
output2 = torch.randn(batch_size, feature_dim)
# 随机生成标签
label = torch.randint(0, 2, (batch_size,)).float()

# 初始化损失函数
criterion = ContrastiveLoss(margin=2.0)
# 计算损失
loss = criterion(output1, output2, label)
print("criterion:",criterion)
print("label:",label)
print("loss:",loss)

criterion: ContrastiveLoss()
output1: tensor([[-0.6402,  1.2064, -0.8874,  0.5344, -0.0905, -1.4215,  0.6936, -1.0155,
          0.8290,  0.0432,  0.0135, -0.5282, -0.6873, -0.1482,  1.2418, -0.6740,
          0.6099,  0.0352, -0.6171,  0.1016,  0.0257,  1.1334, -1.9276,  0.0170,
         -0.0376,  0.4355,  1.0632,  1.3447,  0.8483, -0.3539,  0.8564, -3.2375,
          0.6796,  0.4831, -1.2021,  1.1912,  0.1917, -1.2580,  0.2884, -1.6672,
         -0.6044, -0.2327,  0.2129,  1.4749, -0.1453, -0.5295,  0.7447,  0.2001,
          0.7428, -0.8903,  0.7989, -0.6627,  0.4231, -0.2735,  1.5403,  1.6477,
         -0.5212,  1.5060,  0.0085,  0.0422,  1.8049, -0.0237,  0.4485,  0.7987,
         -1.1675, -1.7204,  0.1865,  1.6418, -0.4754,  1.5662,  0.7594,  0.0690,
          0.2962, -0.4412,  0.1141,  0.8542,  0.0807,  0.1772, -0.0401,  0.8197,
         -0.8345,  0.6284, -0.0995, -1.3038, -0.0057,  0.9973, -0.0974, -0.6298,
          1.8477,  0.8692,  1.8334,  0.8741, -0.3400,  0.7590, -0.4553,